In [1]:
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.mime.base import MIMEBase

import smtplib, ssl

import pickle

In [2]:
# pkl files and data structures
pkl_path = 'pkl\\'

# load the events data structure (if it exists)
events_fn = 'events.pkl'
# verify if the file exists
try:
    with open(pkl_path+events_fn,"rb") as events_in:
        events = pickle.load(events_in)
except FileNotFoundError:
    events = []
print ('There are {0} events!'.format(len(events)))
#print (events)

# load the articles data structure (if it exists)
articles_fn = 'articles.pkl'
# verify if the file exists
try:
    with open(pkl_path+articles_fn,"rb") as articles_in:
        articles = pickle.load(articles_in)
except FileNotFoundError:
    articles = []
print ('There are {0} articles!'.format(len(articles)))
#print (articles)

# load the authors data structure (if it exists)
authors_fn = 'authors.pkl'
# verify if the file exists
try:
    with open(pkl_path+authors_fn,"rb") as authors_in:
        authors = pickle.load(authors_in)
except FileNotFoundError:
    authors = []
print ('There are {0} authors!'.format(len(authors)))
#print (authors)

# load the articles/authors data structure (if it exists)
art_aut_fn = 'art_aut.pkl'
# verify if the file exists
try:
    with open(pkl_path+art_aut_fn,"rb") as art_aut_in:
        art_aut = pickle.load(art_aut_in)
except FileNotFoundError:
    art_aut = []
print ('There are {0} articles-authors relationships!'.format(len(art_aut)))
#print (art_aut)

# load the forms data structure (if it exists)
forms_fn = 'forms.pkl'
# verify if the file exists
try:
    with open(pkl_path+forms_fn,"rb") as forms_in:
        forms = pickle.load(forms_in)
except FileNotFoundError:
    forms = []
print ('There are {0} forms!'.format(len(forms)))




There are 5 events!
There are 162 articles!
There are 341 authors!
There are 506 articles-authors relationships!
There are 321 forms!


In [3]:
# email server (sending) settings

smtp_server_address = "mail.gpcg.pt"
smtp_server_port = 465  # For SSL

# user credentials
username="publicacoes@gpcg.pt"
password="GPCG2020egdl"


In [4]:
# The message sender
sender_email = "publicacoes@gpcg.pt"

# The message destination cc address
cc_email = "publicacoes@gpcg.pt"


In [5]:
#  The message
message_subject = "REMINDER: GPCG - authorization to publish in EG DL ({authorname})"
message1_template = """\
Dear {authorname}, 

author of paper

{title}

published in

{event}

The Eurographics Portuguese Chapter (referred to as GPCG) 
aims to make available in the Eurographics Digital Library all the papers
published in this event - we are currently processing all events from 2003 to 2007.

We made a thorough effort to locate what we believe to be your current email address, 
since the one listed in the paper seems to no longer exist. 

We are sending you this request because we really need our authors collaboration on this process. 
Without your collaboration we will not be able to make available the totality of the publications.

We need the authors authorization, as described in the attached form.
It is enough that one of the authors authorizes. 
We request that author to sign this form and send the respective PDF by email to publicacoes@gpcg.pt

Please send the signed form by replying to this message, without changing the form's filename. 
That will help on automatically keeping track of the large number of replies we are expecting.

You may:

i) sign the file using a digitally certified signature, or

ii) insert into the file an image with your signature, or

iii) print the file, manually sign and digitally scan it back.

Making these publications available in the Eurographics Digital Library will preserve 
GPCG's bibliographic collection and will contribute to increase each publication visibility.

Best regards,

GPCG Direction Board"""

In [6]:
#  The message
messageseveral_template = """\
Dear {authorname}, 

you are the author of several papers published in events promoted
by the Eurographics Portuguese Chapter (referred to as GPCG) from 2003
to 2007.

GPCG aims to make available in the Eurographics Digital Library all the papers
published in these events.

We made a thorough effort to locate what we believe to be your current email address, 
since the one listed in the paper seems to no longer exist. 

We are sending you this request because we really need our authors collaboration on this process. 
Without your collaboration we will not be able to make available the totality of the publications.

We need the authors authorization, as described in the attached form.
It is enough that one of the authors authorizes. 
This authorization will apply to all papers listed in the appendix 
(see the attached "form-app.pdf" file).

We request you to SIGN BOTH THE FORM AND THE APPENDIX 
and send the respective PDFs by email to publicacoes@gpcg.pt

Please send the signed form and appendix by replying to this message, 
without changing the files names. 
That will help on automatically keeping track of the large number of replies we are expecting.

You may:

i) sign both files using a digitally certified signature, or

ii) insert into both files an image with your signature, or

iii) print the files, manually sign and digitally scan them back.

Making these publications available in the Eurographics Digital Library will preserve 
GPCG's bibliographic collection and will contribute to increase each publication visibility.

Best regards,

GPCG Direction Board"""

In [7]:
# Create a secure SSL context
context = ssl.create_default_context()

# login
try:
    server = smtplib.SMTP_SSL(smtp_server_address, smtp_server_port, context=context)
    server.login(username, password)
except:
    print ("Unexpected error creating and logging to SMTP server. Terminate!")
    raise  

In [8]:
# count forms statuses
form_count = {}
for form in forms:

    if form['state'] in form_count:
        form_count[form['state']] += 1
    else:
        form_count[form['state']] = 1
        
print (form_count)

{'auth': 58, 'ignore': 162, 'loaded': 42, 'emailNotFound': 28, 'newEmail': 31}


In [9]:
forms_path = "forms\\"

for n, form in enumerate(forms):
    
    # process ONLY 'newEmail' forms
    if form['state']!='newEmail':
        continue
        
    # locate author
    for author in authors:
        if author['ID']==form['ID']:
            break
                
    # Build the MIME message
    message = MIMEMultipart()
    message["Subject"] = message_subject.format(authorname=author['name'])
    message["From"] = sender_email
    message["Cc"] = cc_email
    message["To"] = author['email']    

    # does this form cover a single article or several ?
    if len(form['articles'])==1:
        
        # locate article
        for article in articles:
            if article['ID']==form['articles'][0]:
                break

        # locate event
        for event in events:
            if article['event']==event['acr']:
                break
        
        message_text = message1_template.format(authorname=author['name'],
                                                title=article['title'],
                                                event=event['name'])
        message.attach(MIMEText(message_text,"plain"))

        # attached the form PDF
        filename = form['file']
        fN = forms_path+filename
        try:
            with open(fN, "rb") as attachment:
                # Add file as application/octet-stream
                # Email client can usually download this automatically as attachment
                part = MIMEBase("application", "octet-stream")
                part.set_payload(attachment.read())
        except FileNotFoundError:
            print ("ERROR: file ", fN, " not found!")
        
        # Encode file in ASCII characters to send by email    
        encoders.encode_base64(part)

        # Add header as key/value pair to attachment part
        part.add_header(
            "Content-Disposition",
            f"attachment; filename= form.pdf",
        )
    
        # Add attachment to message 
        message.attach(part)

        # attachment PDFs DONE
        
    else:  # multiple articles form
        
        message_text = messageseveral_template.format(authorname=author['name'])
        message.attach(MIMEText(message_text,"plain"))
        
        # read the form and form appendix PDF files
        fN = form['file']
        for ii, filename in enumerate([fN,fN.replace("-form.","-form-app.")]):
            # Open PDF file in binary mode
            try:
                with open(forms_path+filename, "rb") as attachment:
                    # Add file as application/octet-stream
                    # Email client can usually download this automatically as attachment
                    part = MIMEBase("application", "octet-stream")
                    part.set_payload(attachment.read())
            except FileNotFoundError:
                print ("ERROR: file ", forms_path+filename, " not found!")
        
            # Encode file in ASCII characters to send by email    
            encoders.encode_base64(part)

            # Add header as key/value pair to attachment part
            part.add_header(
                "Content-Disposition",
                #f"attachment; filename= {filename}",
                f"attachment; filename= form.pdf" if ii==0 else f"attachment; filename= form-app.pdf",

            )
    
            # Add attachment to message 
            message.attach(part)
        # attachment PDFs DONE
    # end of different type of messages
    
    #convert message to string
    str_msg = message.as_string()
    
    # send message
    destination_email = [author['email']]
    destination_email.append(cc_email)
    server.sendmail(sender_email, destination_email, str_msg)
    print ('Sent email for author {0} ({1})'.format(author["name"],n))
                
    # debug
    #print (destination_addresses)
    #print(message["Subject"])
    #print(message_text)
    
    #form['state']='sent'

# all forms processed


Sent email for author Luís Almeida (9)
Sent email for author Fernando Pedro Birra (14)
Sent email for author Manuel Próspero Santos (15)
Sent email for author Nuno Barata (34)
Sent email for author Vítor Cunha (41)
Sent email for author Miguel Leitão (42)
Sent email for author Luis Filipe Garcia (62)
Sent email for author Andréa Poletto Sonza (71)
Sent email for author Francisco dos S. Rebelo (78)
Sent email for author Ernesto Vilar Filgueiras (79)
Sent email for author Marta S. O. Pereira (81)
Sent email for author José A. C.S Reis (82)
Sent email for author Nuno J. Nunes (103)
Sent email for author Luís Romero (104)
Sent email for author Jorge Santiago (105)
Sent email for author Nuno Correia (106)
Sent email for author Carriço Luís (112)
Sent email for author Maria João Silva (115)
Sent email for author Mário J. Silva (118)
Sent email for author José Guilherme Santa Rosa (133)
Sent email for author Alcina Prata (136)
Sent email for author Piet Kommers (137)
Sent email for author Tia

In [10]:
# close the SMTP connection
server.quit()

(221, b'cp29.webserver.pt closing connection')

In [11]:
# update forms in disk
#with open(pkl_path+forms_fn,"wb") as forms_out:
#    pickle.dump(forms, forms_out)

In [12]:
print ("That's all, folks!")

That's all, folks!
